In [ ]:
! pip install -U langchain-nomic langchain-community tiktoken langchainhub chromadb langchain langgraph gpt4all 

In [ ]:
local_llm='llama3'

In [ ]:
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document


# Define the PDF file path
pdf_path = "/content/sample_data/E50.pdf"

# Load data from PDF
loader = PyPDFLoader(pdf_path)
data = loader.load()
print("extracted doc is as follows:", data)

data_in_str = ''

for page in data:
  data_in_str += page.page_content


text_splitter = SpacyTextSplitter.from_tiktoken_encoder(
    chunk_size=100,
    chunk_overlap=10
)

data_splits = text_splitter.split_text(data_in_str)
print("data_splits:", data_splits[1])

doc_splits = [Document(page_content=t) for t in data_splits]

filtered_docs= []
for doc in doc_splits:
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
        clean_metadata = {k: v for k, v in doc.metadata.items() if isinstance(v, (str,int, float, bool))}
        filtered_docs.append(Document(page_content=doc.page_content, metadata=clean_metadata))
print("filtered_docs:", filtered_docs)

vectorstores = Chroma.from_documents(documents=filtered_docs, 
                                     collection_name="rag-chroma",
                                     embedding=GPT4AllEmbeddings(),
                                     )

retriever = vectorstores.as_retriever()

# Retrieving Documents based on user query

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser

llm= ChatOllama(model=local_llm,format="json",temperature=0)

prompt= PromptTemplate(
    templates="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords
    related to the user question, grade it as relevant. It does not need to be a stringent test. The goal is to filter out
    erroneous retrievals. \n Give a binary score 'yes' or 'no' score to indicate whether the documents is relevant to the question.\n
    Provide the binary score as a JSON with a single key 'score' and no preamble or explaination.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {documents} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """, 
    input_variables=["question","documents"],
)

retrieval_gender =prompt | llm | JsonOutputParser()
question="what is section 51 about?"
docs= retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_gender.invoke({"question": question, "documents": doc_txt}))

# Answer generation

In [ ]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

prompt= PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an assisstant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """, input_variables=["question", "documents"],
)

llm= ChatOllama(model=local_llm, temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm | StrOutputParser()

question = "what is section 51 about?"

docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "questions": question})
print(generation)


# Hallucination Grader

In [ ]:
llm= ChatOllama(model=local_llm,format="json",temperature=0)

prompt= PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a grader assessing whether an answer is grounded in / supported by a set of facts. 
    Give a binary score "yes" or "no" score to indicate whether the answer is grounded in / supported be a set of facts.
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation. 
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents}
    \n ------- \n
    Here is the answer: {generation} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """, input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

# Answer Grader

In [ ]:
llm= ChatOllama(model=local_llm,format="json",temperature=0)

prompt= PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a grader assessing whether an answer is useful to resolve a question. 
    Give a binary score "yes" or "no" score to indicate whether the answer is useful to resolve a question.
    Provide the binary score as a JSON with a single key 'score' and no preamble or explanation. 
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """, input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})
